In [ ]:
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers
from tensorflow.keras.layers import Dense, Activation, Flatten, Dropout, BatchNormalization,Embedding
from tensorflow.keras.layers import Conv2D, MaxPooling2D, ReLU, LSTM,Bidirectional,Attention,Concatenate
from tensorflow.keras import regularizers, optimizers,losses
from tensorflow.keras.metrics import Recall,Precision,AUC,TruePositives,TrueNegatives,FalseNegatives,FalsePositives, SpecificityAtSensitivity,SensitivityAtSpecificity
from tensorflow.keras.utils import plot_model
from tensorflow.keras.preprocessing import image
from tensorflow.keras.utils import to_categorical
from tensorflow.python.keras.utils import np_utils
import numpy as np
import pandas as pd
import matplotlib
import seaborn as sns
import sklearn

from imblearn.over_sampling import RandomOverSampler
import imblearn
import matplotlib.pyplot as plt
import time
import os
import sklearn.metrics as m
from glob import glob
from sklearn.preprocessing import StandardScaler
from sklearn import tree
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split


In [ ]:
#classes = {'mel':0 ,'vasc':1,'df':2,'nv':3,'bkl':4,'bcc':5,'ak':6}
classes = {4: ('nv',' melanocytic nevi'),
           6: ('mel','melanoma'),
           2:('bkl','benign keratosis-like lesions'),
           1: ('bcc',' basal cell carcinoma'),
           5: ('vasc',' pyogenic granulomas and hemorrhage'),
           0: ('akiec','Actinic keratoses and intraepithelial carcinomae'),
           3: ('df','dermatofibroma')}

In [ ]:
sns.countplot(x = 'dx', data = tabular_data)
plt.xlabel('Disease', size=12)
plt.ylabel('Frequency', size=16)
plt.title('Frequency Distribution of Classes', size=12)

In [ ]:
y = data['label']
x = data.drop(columns = ['label'])

oversample = RandomOverSampler()
x,y  = oversample.fit_resample(x,y)

y = to_categorical(y)
x= np.array(x).reshape(-1,224,224,3)
print('Shape of X :',x.shape)

In [ ]:
x = (x-np.mean(x))/np.std(x)

X_train, X_test, Y_train, Y_test = train_test_split(x,y, test_size=0.2, random_state=1)
print(X_train.shape , X_test.shape)
print(Y_train.shape , Y_test.shape)

In [ ]:
import tensorflow as tf
IMAGE_SIZE = 224

X_train_resized = tf.image.resize(X_train, (IMAGE_SIZE, IMAGE_SIZE))
X_test_resized = tf.image.resize(X_test, (IMAGE_SIZE, IMAGE_SIZE))

print(X_train_resized.shape, X_test_resized.shape)

In [ ]:
def blockred(inp, filters):
    # Block 1: Inception
    x = Conv2D(filters, 1, activation="relu", padding='same', use_bias=False)(inp)
    x = layers.MaxPooling2D(2)(x)
    x = BatchNormalization(axis=1)(x)

    # Block 2: VGG (just VGG with 3x3 convolutions)
    y = Conv2D(filters, 3, activation="relu", padding='same', use_bias=False)(inp)
    y = Conv2D(filters, 3, activation="relu", padding='same', use_bias=False)(y)
    y = layers.MaxPooling2D(2)(y)
    y = BatchNormalization(axis=1)(y)

    # Block 3: Inception-ResNet
    # Parallel 1x1, 3x3, 5x5 convolutions
    conv1x1 = Conv2D(filters, 1, activation="relu", padding='same', use_bias=False)(inp)

    conv3x3 = Conv2D(filters, 3, activation="relu", padding='same', use_bias=False)(inp)
    conv3x3 = Conv2D(filters, 3, activation="relu", padding='same', use_bias=False)(conv3x3)

    conv5x5 = Conv2D(filters, 5, activation="relu", padding='same', use_bias=False)(inp)
    conv5x5 = Conv2D(filters, 5, activation="relu", padding='same', use_bias=False)(conv5x5)

    # Adding the residual connection (skip connection)
    # Make sure residual input has the same number of filters
    if inp.shape[-1] != filters:
        inp = Conv2D(filters, 1, padding='same', use_bias=False)(inp)

    inception_resnet_output = layers.add([conv1x1, conv3x3, conv5x5, inp])  # Residual connection added
    inception_resnet_output = BatchNormalization(axis=1)(inception_resnet_output)

    # MaxPooling and Batch Normalization
    inception_resnet_output = layers.MaxPooling2D(2)(inception_resnet_output)

    # Ensure that all blocks output the same number of filters
    if inception_resnet_output.shape[-1] != filters:
        inception_resnet_output = Conv2D(filters, 1, padding='same', use_bias=False)(inception_resnet_output)

    # Combine outputs from all blocks
    output1 = layers.add([x, y, inception_resnet_output])

    return output1

In [ ]:
Name= "CNN0"
inputs = keras.Input(shape=(IMAGE_SIZE,IMAGE_SIZE,3), name="img")
x = layers.Conv2D(32, 3, activation="relu")(inputs)

y = Flatten()(x)
y = Dense(256, activation='relu')(y)
y = Dropout(0.5)(y)
y = Dense(64, activation='relu')(y)
y = Dropout(0.5)(y)
outputs=Dense(7, activation='softmax')(y)
model = keras.Model(inputs, outputs, name=Name)
print(model.summary())

In [ ]:
callback = tf.keras.callbacks.ModelCheckpoint(filepath='best_model.keras',
                                                  monitor='val_acc', mode='max',
                                                 verbose=1)

model.compile(optimizer= keras.optimizers.Adam(),
              loss=keras.losses.CategoricalCrossentropy() ,
              metrics=['acc',Recall(),Precision(),AUC(),TruePositives(),TrueNegatives(),FalseNegatives(),FalsePositives()])

#history = model.fit(X_train_resized, Y_train, epochs=5, validation_data=(X_test_resized, Y_test), batch_size=128, callbacks=[callback])

In [ ]:
# Federated learning with homomorphic encryption
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
from tensorflow.keras.optimizers import Adam
import tenseal as ts

def create_tenseal_context():
    poly_modulus_degree = 16384  # Increased polynomial modulus degree
    context = ts.context(
        ts.SCHEME_TYPE.CKKS,
        poly_modulus_degree=poly_modulus_degree,
        coeff_mod_bit_sizes=[60, 40, 40, 40, 40, 60]  # More coefficient moduli
    )
    context.global_scale = 2**40
    context.generate_galois_keys()
    context.generate_relin_keys()

    return context, poly_modulus_degree


# Modified encrypt_weights function to handle larger inputs
def encrypt_weights(context, weights):
    encrypted_weights = []
    for layer in weights:
        # Flatten and split the weights if they're too large
        flat_weights = layer.flatten().tolist()
        max_size = poly_modulus_degree // 2 - 1

        # Split weights into chunks if needed
        if len(flat_weights) > max_size:
            chunks = [flat_weights[i:i + max_size] for i in range(0, len(flat_weights), max_size)]
            encrypted_chunks = [ts.ckks_vector(context, chunk) for chunk in chunks]
            encrypted_weights.append(encrypted_chunks)
        else:
            encrypted_weights.append(ts.ckks_vector(context, flat_weights))
    return encrypted_weights

# Modified decrypt_weights function to handle split weights
def decrypt_weights(context, encrypted_weights, shapes):
    decrypted_weights = []
    for enc_layer, shape in zip(encrypted_weights, shapes):
        if isinstance(enc_layer, list):  # Handle split weights
            decrypted_chunks = []
            for chunk in enc_layer:
                decrypted_chunks.extend(chunk.decrypt())
            decrypted_array = np.array(decrypted_chunks[:np.prod(shape)])
        else:
            decrypted_array = np.array(enc_layer.decrypt())[:np.prod(shape)]
        decrypted_weights.append(decrypted_array.reshape(shape))
    return decrypted_weights

# Define the number of clients and training rounds
NUM_CLIENTS = 5
NUM_ROUNDS = 50
batch_size = 20
epochs = 2

# Create encryption context
tenseal_context, poly_modulus_degree = create_tenseal_context()

# Split the training data among clients
clients = []
for i in range(NUM_CLIENTS):
    client_data = X_train_resized[i * (len(X_train_resized) // NUM_CLIENTS):(i + 1) * (len(X_train_resized) // NUM_CLIENTS)]
    client_labels = Y_train[i * (len(Y_train) // NUM_CLIENTS):(i + 1) * (len(Y_train) // NUM_CLIENTS)]
    clients.append((client_data, client_labels))

# Store accuracy and loss for each federated round
client_train_accuracy = [[] for _ in range(NUM_CLIENTS)]
client_train_loss = [[] for _ in range(NUM_CLIENTS)]
client_val_accuracy = [[] for _ in range(NUM_CLIENTS)]
client_val_loss = [[] for _ in range(NUM_CLIENTS)]
global_train_accuracy = []
global_train_loss = []
global_val_accuracy = []
global_val_loss = []

# Federated learning loop with Homomorphic Encryption
for round_num in range(NUM_ROUNDS):
    print(f"\n🔄 Federated Round {round_num + 1}/{NUM_ROUNDS}")

    # Select a subset of clients
    selected_client_indices = np.random.choice(len(clients), size=int(NUM_CLIENTS * 0.5), replace=False)
    selected_clients = [clients[i] for i in selected_client_indices]

    # Store encrypted client updates
    encrypted_weights_list = []
    train_accuracies = []
    train_losses = []
    val_accuracies = []
    val_losses = []

    for idx, client in enumerate(selected_clients):
        client_model = tf.keras.models.clone_model(model)
        client_model.set_weights(model.get_weights())

        client_model.compile(optimizer=Adam(learning_rate=0.0001),
                             loss='categorical_crossentropy',
                             metrics=['accuracy'])

        steps_per_epoch = int(len(client[0]) / batch_size)

        # Train the client model
        history = client_model.fit(client[0], client[1], batch_size=batch_size, epochs=epochs, verbose=1,
                                   validation_data=(X_test_resized, Y_test))

        # Collect training and validation metrics
        train_accuracies.append(history.history['accuracy'][-1])
        train_losses.append(history.history['loss'][-1])
        val_accuracies.append(history.history['val_accuracy'][-1])
        val_losses.append(history.history['val_loss'][-1])

        # Store per-client metrics
        client_train_accuracy[idx].append(history.history['accuracy'][-1])
        client_train_loss[idx].append(history.history['loss'][-1])
        client_val_accuracy[idx].append(history.history['val_accuracy'][-1])
        client_val_loss[idx].append(history.history['val_loss'][-1])

        # Encrypt client weights before sending them to server
        encrypted_weights = encrypt_weights(tenseal_context, client_model.get_weights())
        encrypted_weights_list.append(encrypted_weights)

    # Perform encrypted aggregation (Federated Averaging)
    aggregated_encrypted_weights = []
    for layer_idx in range(len(encrypted_weights_list[0])):
        if isinstance(encrypted_weights_list[0][layer_idx], list):  # Handle split weights
            layer_aggregated_chunks = []
            for chunk_idx in range(len(encrypted_weights_list[0][layer_idx])):
                chunk_aggregated = encrypted_weights_list[0][layer_idx][chunk_idx]
                for client_idx in range(1, len(encrypted_weights_list)):
                    chunk_aggregated += encrypted_weights_list[client_idx][layer_idx][chunk_idx]
                layer_aggregated_chunks.append(chunk_aggregated.mul(1 / len(encrypted_weights_list)))
            aggregated_encrypted_weights.append(layer_aggregated_chunks)
        else:
            layer_aggregated = encrypted_weights_list[0][layer_idx]
            for client_idx in range(1, len(encrypted_weights_list)):
                layer_aggregated += encrypted_weights_list[client_idx][layer_idx]
            aggregated_encrypted_weights.append(layer_aggregated.mul(1 / len(encrypted_weights_list)))

    # Decrypt aggregated weights and update global model
    model_shapes = [layer.shape for layer in model.get_weights()]
    new_weights = decrypt_weights(tenseal_context, aggregated_encrypted_weights, model_shapes)
    model.set_weights(new_weights)

    # Store round-wise global metrics
    global_train_accuracy.append(np.mean(train_accuracies))
    global_train_loss.append(np.mean(train_losses))
    global_val_accuracy.append(np.mean(val_accuracies))
    global_val_loss.append(np.mean(val_losses))

# 🎯 Final Evaluation
y_pred = model.predict(X_test_resized)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true = np.argmax(Y_test, axis=1)

# 🏆 Final Global Model Accuracy
from sklearn.metrics import accuracy_score
final_accuracy = accuracy_score(y_true, y_pred_classes)
print(f"\n✅ Final Global Model Accuracy: {final_accuracy * 100:.2f}%")

# 💾 Save Model
model.save('Homo_Encrypt_federated_modelc5r50.h5')
print("\n✅ Encrypted Federated Model saved successfully!")

# 📊 Plot Global Training & Validation Curves
fl_rounds = range(1, NUM_ROUNDS + 1)

plt.figure(figsize=(20, 5))

# Accuracy Plot (Training and Validation)
plt.subplot(1, 2, 1)
plt.plot(fl_rounds, global_train_accuracy, label="Training", color='blue', marker='o', linewidth=2)
plt.plot(fl_rounds, global_val_accuracy, label="Validation", color='green', marker='s', linewidth=2)
plt.xlabel("Communication Rounds")
plt.ylabel("Accuracy")
plt.title("Global Model Accuracy (with Encrypted Aggregation)")
plt.legend()
plt.grid()

# Loss Plot (Training and Validation)
plt.subplot(1, 2, 2)
plt.plot(fl_rounds, global_train_loss, label="Training", color='red', marker='o', linewidth=2)
plt.plot(fl_rounds, global_val_loss, label="Validation", color='orange', marker='s', linewidth=2)
plt.xlabel("Communication Rounds")
plt.ylabel("Loss")
plt.title("Global Model Loss (with Encrypted Aggregation)")
plt.legend()
plt.grid()

plt.show()

In [ ]:
res = model.evaluate(X_test_resized, Y_test, verbose=1)
print(res)

In [ ]:
from sklearn.metrics import confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt

# Define class names
class_names = ['akiec', 'bcc', 'bkl', 'df', 'nv', 'mel', 'vasc']

# Predict the values from the test dataset
Y_pred = model.predict(X_test_resized)
Y_pred_classes = np.argmax(Y_pred, axis=1)
Y_true = np.argmax(Y_test, axis=1)

# Compute the confusion matrix
cm = confusion_matrix(Y_true, Y_pred_classes)

# Plot the confusion matrix
plt.figure(figsize=(10,7))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=class_names, yticklabels=class_names)
plt.xlabel('Predicted')
plt.ylabel('True')
plt.title('Confusion Matrix')
plt.show()
